# Notebook 06 : Interprétabilité avec SHAP

**Projet 7 - Prêt à dépenser : Scoring Crédit**

## Objectifs

1. Charger le pipeline et les données de validation
2. Aligner les features avec le modèle
3. Initialiser SHAP TreeExplainer
4. Calculer les valeurs SHAP
5. Créer des visualisations globales :
 - Summary plot (beeswarm)
 - Bar plot de l'importance
 - Dependence plots
6. Créer des visualisations locales :
 - Waterfall plots pour des clients spécifiques
 - Force plots individuels
7. Analyser l'impact des features principales
8. Comparer des cas de crédit accordé vs refusé

---

## 1. Imports et Configuration

In [ ]:
# Imports standards
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
from pathlib import Path

# SHAP
import shap

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Pour les notebooks
shap.initjs()

print(" Imports réussis")
print(f"Version SHAP: {shap.__version__}")

## 2. Chargement du Pipeline et des Données

In [ ]:
# Chemins
ARTIFACTS_DIR = Path('../artifacts')
FIGURES_DIR = Path('../figures')
FIGURES_DIR.mkdir(exist_ok=True) # Créer le dossier s'il n'existe pas

PIPELINE_PATH = ARTIFACTS_DIR / 'pipeline_final.joblib'
DATA_SPLIT_PATH = ARTIFACTS_DIR / 'data_split.joblib'
FEATURE_NAMES_PATH = ARTIFACTS_DIR / 'feature_names.joblib'

print("="*60)
print("CHARGEMENT DES DONNÉES")
print("="*60)

# Charger le pipeline
print("\nChargement du pipeline...")
pipeline = joblib.load(PIPELINE_PATH)
print(f" Pipeline chargé: {type(pipeline)}")

# Charger les données de validation (on utilisera X_valid comme données de test)
print("\nChargement des données...")
X_train, X_valid, y_train, y_valid = joblib.load(DATA_SPLIT_PATH)
print(f" X_valid shape: {X_valid.shape}")
print(f" y_valid shape: {y_valid.shape}")

# Charger les noms de features si disponibles
try:
 feature_names = joblib.load(FEATURE_NAMES_PATH)
 print(f"\n {len(feature_names)} noms de features chargés")
except FileNotFoundError:
 feature_names = list(X_valid.columns)
 print(f"\n feature_names.joblib non trouvé, utilisation des colonnes de X_valid")
 print(f" {len(feature_names)} features")

print("\n" + "="*60)
print("RÉSUMÉ")
print("="*60)
print(f" - Échantillon de validation: {X_valid.shape[0]:,} clients")
print(f" - Features: {X_valid.shape[1]}")
print(f" - Taux de défaut: {y_valid.mean():.2%}")
print("="*60)

## 3. Alignement des Features avec le Modèle

Important : Il faut s'assurer que les données ont le même nombre de colonnes que lors de l'entraînement.

In [ ]:
print("="*60)
print("ALIGNEMENT DES FEATURES")
print("="*60)

# Récupérer les features attendues par le modèle
try:
 expected_features = pipeline.feature_name_
 print(" Features récupérées via feature_name_")
except:
 try:
 expected_features = pipeline.booster_.feature_name()
 print(" Features récupérées via booster_.feature_name()")
 except:
 try:
 expected_features = pipeline._Booster.feature_name()
 print(" Features récupérées via _Booster.feature_name()")
 except:
 # Si impossible de récupérer, utiliser les feature_names chargés
 expected_features = feature_names
 print(" Utilisation des feature_names chargés")

print(f"\nFeatures attendues par le modèle: {len(expected_features)}")
print(f"Features dans X_valid: {len(X_valid.columns)}")

# Identifier les différences
current_cols = set(X_valid.columns)
expected_cols = set(expected_features)

missing_cols = list(expected_cols - current_cols)
extra_cols = list(current_cols - expected_cols)

if len(missing_cols) > 0:
 print(f"\n Colonnes manquantes: {len(missing_cols)}")
 if len(missing_cols) <= 10:
 print(f" {missing_cols}")
 else:
 print(f" {missing_cols[:5]}... (+{len(missing_cols)-5} autres)")
 
 # Ajouter les colonnes manquantes
 print("\nAjout des colonnes manquantes...")
 missing_df = pd.DataFrame(0, index=X_valid.index, columns=missing_cols)
 X_valid = pd.concat([X_valid, missing_df], axis=1)
 print(" Colonnes ajoutées")

if len(extra_cols) > 0:
 print(f"\n Colonnes en trop: {len(extra_cols)}")
 if len(extra_cols) <= 10:
 print(f" {extra_cols}")
 else:
 print(f" {extra_cols[:5]}... (+{len(extra_cols)-5} autres)")
 
 # Supprimer les colonnes en trop
 print("\nSuppression des colonnes en trop...")
 X_valid = X_valid.drop(columns=extra_cols)
 print(" Colonnes supprimées")

# Réordonner les colonnes
print("\nRéordonnancement des colonnes...")
X_valid = X_valid[expected_features]
print(" Colonnes réordonnées")

print(f"\n Shape finale: {X_valid.shape}")

## 4. Conversion des Types de Données

In [ ]:
print("="*60)
print("CONVERSION DES TYPES")
print("="*60)

# Identifier les colonnes object
object_cols = X_valid.select_dtypes(include=['object']).columns.tolist()
print(f"\nColonnes 'object' à convertir: {len(object_cols)}")

# Convertir les colonnes object en numérique
if len(object_cols) > 0:
 print("Conversion en cours...")
 for col in object_cols:
 X_valid[col] = pd.to_numeric(X_valid[col], errors='coerce')
 print(" Conversion terminée")

# Gérer les valeurs manquantes
nan_count = X_valid.isna().sum().sum()
if nan_count > 0:
 print(f"\nValeurs NaN détectées: {nan_count:,}")
 X_valid = X_valid.fillna(0)
 print(" NaN remplacés par 0")

print(f"\n Données prêtes pour SHAP")
print(f" Shape: {X_valid.shape}")
print(f" NaN restants: {X_valid.isna().sum().sum()}")

## 5. Préparation de l'Échantillon pour SHAP

Pour des raisons de performance, on utilise un échantillon des données.

In [ ]:
print("="*60)
print("PRÉPARATION DE L'ÉCHANTILLON SHAP")
print("="*60)

# Définir la taille de l'échantillon
# Pour SHAP, on recommande 100-500 observations pour un bon compromis vitesse/qualité
sample_size = min(200, len(X_valid))

# Échantillonnage stratifié pour garder la distribution des classes
from sklearn.model_selection import train_test_split

if len(X_valid) > sample_size:
 X_sample, _, y_sample, _ = train_test_split(
 X_valid, y_valid, 
 train_size=sample_size,
 stratify=y_valid,
 random_state=42
 )
 print(f" Échantillon stratifié créé: {sample_size} observations")
else:
 X_sample = X_valid
 y_sample = y_valid
 print(f" Utilisation de toutes les données: {len(X_sample)} observations")

print(f"\nDistribution dans l'échantillon:")
print(f" Classe 0 (bon): {(y_sample == 0).sum()} ({(y_sample == 0).mean():.1%})")
print(f" Classe 1 (défaut): {(y_sample == 1).sum()} ({(y_sample == 1).mean():.1%})")

# Générer les prédictions pour l'échantillon
print("\nGénération des prédictions...")
y_pred_proba = pipeline.predict_proba(X_sample)[:, 1]
print(f" Prédictions générées")
print(f" Probabilité moyenne: {y_pred_proba.mean():.3f}")
print(f" Min: {y_pred_proba.min():.3f}, Max: {y_pred_proba.max():.3f}")

## 6. Initialisation de SHAP TreeExplainer

Pour les modèles LightGBM, on utilise **TreeExplainer** qui est optimisé et rapide.

In [ ]:
print("="*60)
print("INITIALISATION DE SHAP")
print("="*60)

# Initialiser l'explainer avec le modèle
print("\nInitialisation du SHAP TreeExplainer...")
explainer = shap.TreeExplainer(pipeline)
print(f" Explainer initialisé pour {type(pipeline).__name__}")

# Calculer les valeurs SHAP
print(f"\nCalcul des valeurs SHAP pour {len(X_sample)} observations...")
print(" Cela peut prendre quelques secondes...")

shap_values = explainer.shap_values(X_sample)

# Pour classification binaire, shap_values peut être:
# - Une liste [class_0, class_1] pour certains modèles
# - Un array unique pour d'autres
if isinstance(shap_values, list):
 # Prendre les valeurs pour la classe positive (défaut = 1)
 shap_values_class1 = shap_values[1]
 print(" Valeurs SHAP calculées (liste - pris classe 1)")
else:
 shap_values_class1 = shap_values
 print(" Valeurs SHAP calculées (array unique)")

print(f"\nShape des valeurs SHAP: {shap_values_class1.shape}")
print(f"Expected value (base): {explainer.expected_value if not isinstance(explainer.expected_value, list) else explainer.expected_value[1]:.4f}")

# Stocker la base value
if isinstance(explainer.expected_value, list):
 base_value = explainer.expected_value[1]
else:
 base_value = explainer.expected_value

print(f"\n La base value ({base_value:.4f}) représente la prédiction moyenne du modèle")

## 7. Visualisations Globales

### 7.1 Summary Plot (Beeswarm)

Ce graphique montre :
- **Axe vertical** : Features ordonnées par importance
- **Axe horizontal** : Impact sur la prédiction
- **Couleur** : Valeur de la feature (rouge = élevé, bleu = faible)

In [ ]:
print("="*60)
print("SUMMARY PLOT (BEESWARM)")
print("="*60)

plt.figure(figsize=(14, 10))
shap.summary_plot(
 shap_values_class1,
 X_sample,
 max_display=20,
 show=False
)
plt.title("Impact des Features sur la Probabilité de Défaut\n(SHAP Summary Plot)", 
 fontsize=16, fontweight='bold', pad=20)
plt.xlabel("Impact SHAP sur la prédiction", fontsize=12)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'shap_summary_beeswarm.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Summary plot créé et sauvegardé")
print(f" Fichier: {FIGURES_DIR / 'shap_summary_beeswarm.png'}")

### 7.2 Bar Plot (Importance Moyenne Absolue)

Ce graphique montre l'importance moyenne absolue de chaque feature.

In [ ]:
print("="*60)
print("BAR PLOT (IMPORTANCE)")
print("="*60)

plt.figure(figsize=(14, 10))
shap.summary_plot(
 shap_values_class1,
 X_sample,
 max_display=20,
 plot_type='bar',
 show=False
)
plt.title("Importance Moyenne des Features\n(SHAP Bar Plot)", 
 fontsize=16, fontweight='bold', pad=20)
plt.xlabel("Importance moyenne (|SHAP value|)", fontsize=12)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'shap_bar_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Bar plot créé et sauvegardé")
print(f" Fichier: {FIGURES_DIR / 'shap_bar_plot.png'}")

### 7.3 Top Features

Calculons les features les plus importantes selon SHAP.

In [ ]:
print("="*60)
print("IMPORTANCE DES FEATURES")
print("="*60)

# Calculer l'importance moyenne absolue
feature_importance_shap = pd.DataFrame({
 'feature': X_sample.columns,
 'importance': np.abs(shap_values_class1).mean(axis=0)
}).sort_values('importance', ascending=False)

print("\nTop 15 Features les plus importantes (SHAP):")
print("="*60)
for idx, row in feature_importance_shap.head(15).iterrows():
 print(f"{row['feature']:50s} : {row['importance']:.6f}")

# Sauvegarder
feature_importance_shap.to_csv(ARTIFACTS_DIR / 'shap_feature_importance.csv', index=False)
print(f"\n Importance SHAP sauvegardée")
print(f" Fichier: {ARTIFACTS_DIR / 'shap_feature_importance.csv'}")

### 7.4 Dependence Plots

Ces graphiques montrent comment la valeur d'une feature influence la prédiction.

In [ ]:
print("="*60)
print("DEPENDENCE PLOTS")
print("="*60)

# Prendre les 6 features les plus importantes
top_features = feature_importance_shap.head(6)['feature'].tolist()

print(f"\nCréation des dependence plots pour les {len(top_features)} features les plus importantes...")

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

for idx, feature_name in enumerate(top_features):
 # Trouver l'index de la feature
 feature_idx = list(X_sample.columns).index(feature_name)
 
 plt.sca(axes[idx])
 shap.dependence_plot(
 feature_idx,
 shap_values_class1,
 X_sample,
 show=False,
 ax=axes[idx]
 )
 axes[idx].set_title(f"Dependence: {feature_name}", fontsize=13, fontweight='bold')
 axes[idx].set_xlabel(feature_name, fontsize=11)
 axes[idx].set_ylabel("Impact SHAP", fontsize=11)

plt.suptitle("Dependence Plots - Top 6 Features", fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'shap_dependence_plots.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Dependence plots créés et sauvegardés")
print(f" Fichier: {FIGURES_DIR / 'shap_dependence_plots.png'}")

## 8. Visualisations Locales (Clients Individuels)

### 8.1 Sélection d'Exemples Intéressants

In [ ]:
print("="*60)
print("SÉLECTION D'EXEMPLES")
print("="*60)

# Trouver des exemples intéressants:
# 1. Client avec très faible risque (crédit accordé)
idx_low_risk = np.argmin(y_pred_proba)

# 2. Client avec risque élevé (crédit refusé)
idx_high_risk = np.argmax(y_pred_proba)

# 3. Client proche du seuil de décision
# Charger le seuil optimal si disponible
try:
 params = joblib.load(ARTIFACTS_DIR / 'parametres_decision.joblib')
 seuil = params['seuil_optimal']
 print(f" Seuil optimal chargé: {seuil:.4f}")
except:
 seuil = 0.5
 print(f" Utilisation du seuil par défaut: {seuil}")

idx_borderline = np.argmin(np.abs(y_pred_proba - seuil))

# Afficher les exemples sélectionnés
print("\nExemples sélectionnés:")
print("="*60)

exemples = [
 (" FAIBLE RISQUE", idx_low_risk),
 (" BORDERLINE", idx_borderline),
 (" RISQUE ÉLEVÉ", idx_high_risk)
]

for nom, idx in exemples:
 proba = y_pred_proba[idx]
 vrai_label = "Défaut" if y_sample.iloc[idx] == 1 else "Bon"
 decision = "REFUSÉ" if proba >= seuil else "ACCEPTÉ"
 
 print(f"\n{nom} (Index: {idx})")
 print(f" Probabilité de défaut: {proba:.4f}")
 print(f" Vraie classe: {vrai_label}")
 print(f" Décision: {decision}")

### 8.2 Waterfall Plots

Les waterfall plots montrent comment chaque feature contribue à la prédiction pour un client spécifique.

In [ ]:
print("="*60)
print("WATERFALL PLOTS")
print("="*60)

# Créer un waterfall plot pour chaque exemple
for nom, idx in exemples:
 print(f"\nCréation du waterfall plot pour: {nom}")
 
 plt.figure(figsize=(12, 8))
 
 # Créer l'explication
 explanation = shap.Explanation(
 values=shap_values_class1[idx],
 base_values=base_value,
 data=X_sample.iloc[idx].values,
 feature_names=X_sample.columns.tolist()
 )
 
 shap.waterfall_plot(explanation, max_display=15, show=False)
 
 plt.title(f"Waterfall Plot - {nom}\nProbabilité: {y_pred_proba[idx]:.4f}", 
 fontsize=14, fontweight='bold', pad=20)
 plt.tight_layout()
 
 # Sauvegarder
 filename = f"shap_waterfall_{nom.split()[1].lower()}.png"
 plt.savefig(FIGURES_DIR / filename, dpi=300, bbox_inches='tight')
 plt.show()
 
 print(f" Sauvegardé: {filename}")

print("\n Tous les waterfall plots créés")

### 8.3 Force Plots

Les force plots montrent visuellement comment les features poussent la prédiction vers le haut ou vers le bas.

In [ ]:
print("="*60)
print("FORCE PLOTS")
print("="*60)

# Force plot pour chaque exemple
for nom, idx in exemples:
 print(f"\nCréation du force plot pour: {nom}")
 
 # Force plot
 shap.force_plot(
 base_value,
 shap_values_class1[idx],
 X_sample.iloc[idx],
 matplotlib=True,
 show=False,
 figsize=(20, 3)
 )
 
 plt.title(f"Force Plot - {nom} (Proba: {y_pred_proba[idx]:.4f})", 
 fontsize=12, fontweight='bold')
 plt.tight_layout()
 
 # Sauvegarder
 filename = f"shap_force_{nom.split()[1].lower()}.png"
 plt.savefig(FIGURES_DIR / filename, dpi=300, bbox_inches='tight')
 plt.show()
 
 print(f" Sauvegardé: {filename}")

print("\n Tous les force plots créés")

## 9. Analyse Comparative

Comparons les features qui influencent les décisions pour les clients acceptés vs refusés.

In [ ]:
print("="*60)
print("ANALYSE COMPARATIVE")
print("="*60)

# Séparer les clients selon la décision
accepted = y_pred_proba < seuil
refused = y_pred_proba >= seuil

print(f"\nDistribution des décisions:")
print(f" Acceptés: {accepted.sum()} ({accepted.mean():.1%})")
print(f" Refusés: {refused.sum()} ({refused.mean():.1%})")

# Calculer l'importance moyenne pour chaque groupe
shap_accepted = np.abs(shap_values_class1[accepted]).mean(axis=0)
shap_refused = np.abs(shap_values_class1[refused]).mean(axis=0)

# Créer un DataFrame comparatif
comparison_df = pd.DataFrame({
 'feature': X_sample.columns,
 'importance_accepted': shap_accepted,
 'importance_refused': shap_refused,
 'difference': shap_refused - shap_accepted
}).sort_values('difference', ascending=False)

print("\nTop 10 features plus importantes pour les REFUSÉS:")
print("="*60)
for idx, row in comparison_df.head(10).iterrows():
 print(f"{row['feature']:45s} : +{row['difference']:.6f}")

print("\nTop 10 features plus importantes pour les ACCEPTÉS:")
print("="*60)
for idx, row in comparison_df.tail(10).iterrows():
 print(f"{row['feature']:45s} : {row['difference']:.6f}")

# Sauvegarder
comparison_df.to_csv(ARTIFACTS_DIR / 'shap_comparison_accepted_refused.csv', index=False)
print(f"\n Analyse comparative sauvegardée")
print(f" Fichier: {ARTIFACTS_DIR / 'shap_comparison_accepted_refused.csv'}")

## 10. Résumé et Conclusions

In [ ]:
print("="*60)
print("RÉSUMÉ DE L'ANALYSE SHAP")
print("="*60)

print("\n VISUALISATIONS CRÉÉES:")
print(" Summary plot (beeswarm)")
print(" Bar plot (importance)")
print(" Dependence plots (top 6 features)")
print(" Waterfall plots (3 exemples)")
print(" Force plots (3 exemples)")

print("\n FICHIERS SAUVEGARDÉS:")
print(f" Figures: {FIGURES_DIR}/")
print(f" Importance SHAP: {ARTIFACTS_DIR / 'shap_feature_importance.csv'}")
print(f" Comparaison: {ARTIFACTS_DIR / 'shap_comparison_accepted_refused.csv'}")

print("\n TOP 5 FEATURES LES PLUS IMPORTANTES:")
for idx, row in feature_importance_shap.head(5).iterrows():
 print(f" {idx+1}. {row['feature']}")

print("\n INSIGHTS CLÉS:")
print(" - Les valeurs SHAP permettent d'expliquer chaque prédiction")
print(" - Les features importantes varient selon le profil (accepté/refusé)")
print(" - Les visualisations peuvent être utilisées pour le dashboard")

print("\n" + "="*60)
print("ANALYSE SHAP TERMINÉE")
print("="*60)

In [ ]:
#!/usr/bin/env python3
"""
Script d'extraction des figures - Notebook 06
SHAP Interprétabilité (waterfall, summary, force plots)
"""

import json
import base64
import os
from pathlib import Path

def extract_figures_from_notebook(notebook_path, output_dir):
 """
 Extrait toutes les figures SHAP du notebook d'interprétabilité
 """
 # Créer le dossier de sortie
 output_dir = Path(output_dir)
 output_dir.mkdir(parents=True, exist_ok=True)
 
 # Charger le notebook
 with open(notebook_path, 'r', encoding='utf-8') as f:
 notebook = json.load(f)
 
 figure_count = 0
 
 # Parcourir toutes les cellules
 for cell_idx, cell in enumerate(notebook['cells']):
 # Chercher les cellules avec outputs
 if cell['cell_type'] == 'code' and 'outputs' in cell:
 for output in cell['outputs']:
 # Chercher les images PNG
 if 'data' in output and 'image/png' in output['data']:
 figure_count += 1
 
 # Décoder l'image base64
 img_data = output['data']['image/png']
 img_bytes = base64.b64decode(img_data)
 
 # Déterminer le nom du fichier selon le contexte
 cell_source = ''.join(cell['source']).lower()
 
 if 'waterfall' in cell_source:
 filename = f'nb06_fig{figure_count:02d}_shap_waterfall.png'
 elif 'summary' in cell_source and 'plot' in cell_source:
 filename = f'nb06_fig{figure_count:02d}_shap_summary.png'
 elif 'force' in cell_source:
 filename = f'nb06_fig{figure_count:02d}_shap_force.png'
 elif 'dependence' in cell_source or 'dépendance' in cell_source:
 filename = f'nb06_fig{figure_count:02d}_shap_dependence.png'
 elif 'bar' in cell_source or 'importance' in cell_source:
 filename = f'nb06_fig{figure_count:02d}_shap_importance.png'
 else:
 filename = f'nb06_fig{figure_count:02d}_shap_figure.png'
 
 # Sauvegarder l'image
 output_path = output_dir / filename
 with open(output_path, 'wb') as img_file:
 img_file.write(img_bytes)
 
 print(f" Extrait : {filename}")
 
 print(f"\n Total : {figure_count} figures extraites du Notebook 06")
 return figure_count


if __name__ == '__main__':
 # Chemins
 NOTEBOOK_PATH = 'Barre_Stephane_P7_06_interpretabilite_shap.ipynb'
 OUTPUT_DIR = 'outputs/figures_p7/notebook_06'
 
 # Extraction
 print(" Extraction des figures du Notebook 06 (SHAP)...")
 print(f" Notebook : {NOTEBOOK_PATH}")
 print(f" Sortie : {OUTPUT_DIR}\n")
 
 extract_figures_from_notebook(NOTEBOOK_PATH, OUTPUT_DIR)